In [1]:
import os
import tensorflow_model_analysis as tfma
from modules.merge_dataset import merge_dataset
from tfx import components
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

In [2]:
merge_dataset("data/movie.csv", "data/rating.csv")

'Merge dataset success'

In [3]:
PIPELINE_ROOT = "pipeline"
SCHEMA_PIPELINE_NAME = "movies-rec"
SERVING_MODEL_DIR = "serving_model_dir"

In [4]:
DATA_ROOT = "data/merge"

In [5]:
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [6]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2),
    ])
)

example_gen = components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output
)
context.run(example_gen)

In [ ]:
statistics_gen = components.StatisticsGen(
    examples=example_gen.outputs["examples"]
)
context.run(statistics_gen)

In [ ]:
context.show(statistics_gen.outputs["statistics"])

In [ ]:
schema_gen = components.SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
context.run(schema_gen)

In [ ]:
example_validator = components.ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
context.run(example_validator)

In [ ]:
context.show(example_validator.outputs["anomalies"])


In [ ]:
TRANSFORM_MODULE_FILE = "modules/transform.py"

In [ ]:
transform = components.Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
context.run(transform)

In [ ]:
context.show(transform.outputs["post_transform_schema"])

In [ ]:
TRAINER_MODULE_FILE = "modules/trainer.py"

In [ ]:
trainer = components.Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=transform.outputs["post_transform_schema"],
    train_args=trainer_pb2.TrainArgs(
        splits=["train"], 
        num_steps=2000,
    ),
    eval_args=trainer_pb2.EvalArgs(
        splits=["eval"],
        num_steps=1200,
    ),
    custom_config={"epochs": 5}
)

context.run(trainer)

In [ ]:
pusher = components.Pusher(
    model=trainer.outputs["model"],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=os.path.join(
                SERVING_MODEL_DIR, "movie-recommender"
            ),
        )
    )
)

context.run(pusher)